In [ ]:
%load_ext autoreload
%matplotlib widget
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tqdm.autonotebook import trange
from keras.datasets import mnist

%autoreload 1
%aimport jsi

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# 2x2 jsi generation around the exceptional point
nodes = 2
y1 = 0.7 
y2 = 0.2
y_ex = 0.3
g = 0.03
j = 0.2575

try:
    for fig in figs:
        plt.close(fig)
except:
    pass
    
figs = []
for i in np.linspace(-0.25, 0.25, 7):
    input = {
        'nodes': nodes, 
        'orth_itr': 5,
        'js': np.array([[j]], dtype=np.complex64),
        'jr': np.array([np.repeat(y_ex, 4)], dtype=np.complex64),
        'g': np.array([g], dtype=np.complex64), 
        # 'y0s': np.array([[y1, y2, y1, y2]], dtype=np.complex64)
        'y0s': np.array([[y1 - i, y2 + i, y1 - i, y2 + i]], dtype=np.complex64)
    }
    pred, loss, output = jsi.jsi_backprop(input, None, train=False)
    print("i = " + str(i))
    print("max: " + str(np.max(output)))
    print()
    figs.append( jsi.pltSect(output, 0, 0, nodes, nodes) )

In [ ]:
# 3x3 jsi generation
n=10
nodes = 3
g = 0.03
for i in np.linspace(0, n, n+1):
    data = {
        'nodes': nodes, 
        'orth_itr': 1, 
        'js': np.array([[0.5*np.exp(1j * i / (n * 2 * np.pi)), 0]], dtype=np.complex64), 
        'jr': np.array([[0.1, 0.5, 0.1, 0.1, 0.5, 0.1]], dtype=np.complex64), 
        'g': np.array([g], dtype=np.complex64), 
        'y0s': np.array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]], dtype=np.complex64) 
    }
    pred, loss, output = jsi.jsi_backprop(data, None, train=False)
    fig=jsi.pltSect(output, 0, 0, nodes, nodes)

In [ ]:
# 1-node accuracy test against analytical result

prediction = []
actual = []
n = 100
for i in np.linspace(0, 1, n):
    y_ex = np.random.uniform()
    y_0 = np.random.uniform()
    g = 0.03
    y_t = (y_ex + y_0)

    data = {
        'nodes': 1, 
        'orth_itr': 1,
        'js': np.array([[]], dtype=np.complex64),
        'jr': np.array([[y_ex, y_ex]], dtype=np.complex64), 
        'g': np.array([g], dtype=np.complex64), 
        'y0s': np.array([[y_0, y_0]], dtype=np.complex64)
    }
    pred, loss, output = jsi.jsi_backprop(data, None, train=False)
    prediction.append(output[0, 0].real)
    delta = 1 / (np.power(y_t / 2, 2) - np.power(g, 2))
    act = 4 * np.pi * np.pi * y_ex * y_ex * g * g * np.power(delta, 2) * (1 - np.power(y_t, 2) / 2 * delta) * (1 - np.power(y_t, 2) / 2 * delta)
    actual.append(act)
plt.clf()
# plt.scatter(range(n), prediction, c='red', s=0.3)
# plt.scatter(range(n), actual, c='blue', s=0.3)
plt.scatter(range(n), np.array(prediction) / np.array(actual), c='green', s=0.5)
plt.show()